In [1]:
import cv2
import numpy as np
import glob
import random
import time 

In [2]:
load_dir = "C:/Users/Dell/DataScience/Lums/CS5310/MiniProject/Task4"

In [3]:
# Load Yolo
yolo_net = cv2.dnn.readNet( load_dir + "/weights3hrs/mask-yolov3_10000.weights", load_dir + "/weights3hrs/mask-yolov3.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]

In [4]:
classes = ["Person", "mask", "noMask"]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

colors = [0, 122, 255] # color of boxes
# print(colors)
# print(len(colors))

In [5]:
cameras = {
    "Camera-1": load_dir + "/Videos/campus4-c0.mp4",
    "Camera-2": load_dir + "/Videos/campus4-c1.mp4",
    "Camera-3": load_dir + "/Videos/campus4-c2.mp4"    
}

In [6]:
Homography_matrix = {}
    
Homography_matrix["Camera-1"] = np.array([[-0.211332, -0.405226, 70.781223], [-0.019746, -1.564936, 226.377280], [-0.000025, -0.001961, 0.160791]])
Homography_matrix["Camera-2"] = np.array([[0.000745, 0.350335, -98.376103], [-0.164871, -0.390422, 54.081423], [0.000021, -0.001668, 0.111075]])
Homography_matrix["Camera-3"] = np.array([[0.089976, 1.066795, -152.055667], [-0.116343, 0.861342, -75.122116], [0.000015, 0.001442, -0.064065]])

In [7]:
def Get_Available_Cams(cams_detail,  starting = 0):
    available_cams = {}

    for key, video_address in cams_detail.items():
        cam = cv2.VideoCapture(video_address)
        if cam.isOpened() == True:
            cam.set(1,starting)
            available_cams[key] = cam
            
    return available_cams

In [8]:
def Apply_Homograpy_On_Frames(current_frame):
    merge_image = []
    flag = True    
    for key in list(current_frame.keys()):
        new_image = cv2.warpPerspective(current_frame[key], Homography_matrix[key] ,(600,600)).astype('uint32')        
        if flag == True:
            merge_image = new_image
            flag = False
        else:
            merge_image += new_image
        
    length = len(current_frame.keys())
    merge_image =  np.array((np.array(merge_image) / length)).astype('uint8')
    return merge_image

In [9]:
def Object_Detection(current_frame):
    
    object_detections = {}
    
    for key in list(current_frame.keys()):
        
        begin = time.time()
        # Image
        img = current_frame[key]
        height, width, channels = img.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

        yolo_net.setInput(blob)
        outs = yolo_net.forward(output_layers)

        # Showing informations on the screen
        detection_data = {"Boxes":[], "ClassIDs": [], "Confidences": [], "Center": [] }
        
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.4:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    detection_data["Boxes"].append([x, y, w, h])
                    detection_data["ClassIDs"].append(class_id)
                    detection_data["Confidences"].append(float(confidence))
#                     detection_data["Center"].append([[center_x], [center_y ], [1]])
                    detection_data["Center"].append([center_x, center_y+ int(h/2)])
#                     detection_data["Center"].append([center_x, center_y])
    
        
        object_detections[key] = detection_data
    
    show_bounding_boxes(current_frame, object_detections)
    return object_detections

In [10]:
def show_bounding_boxes(current_frame, object_detections):

    for key in current_frame.keys():
        img = current_frame[key]
        detection_data = object_detections[key]
        
        indexes = cv2.dnn.NMSBoxes(detection_data["Boxes"], detection_data["Confidences"], 0.5, 0.4)
        font = cv2.ACCESS_MASK
        
        for i in range(len(detection_data["Boxes"])):
            if i in indexes:
                x, y, w, h = detection_data["Boxes"][i]
                label = str(classes[detection_data["ClassIDs"][i]]) + str(np.around(detection_data["Confidences"][i],2)*100) + "%"
                color = colors[detection_data["ClassIDs"][i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
                cv2.putText(img, label , (x, y-5), font, 0.4, color, 1)

        cv2.imshow(key, img)
        cv2.waitKey(1)

In [11]:
def show_point_top_view(new_locations):
    path_top_image = load_dir + "/Images/top_view.jpg"
    new_image = cv2.imread(path_top_image, 1)
    for points in new_locations:
        new_image = cv2.circle(new_image, (points[0], points[1]), 5, (255, 0, 0), 5)
        
    cv2.imshow("Top View ", new_image)
    cv2.waitKey(1)

In [12]:
def label_on_top_view(object_detections, top_view_frame):
    
    new_image = top_view_frame
    new_locations = []
    booleanV = True
    for key in object_detections:
        detection_data = object_detections[key]
        indexes = cv2.dnn.NMSBoxes(detection_data["Boxes"], detection_data["Confidences"], 0.5, 0.4)
        
        for i in range(len(detection_data["Boxes"])):
            if i in indexes:
                center = detection_data["Center"][i]                
                new_location =  np.dot(Homography_matrix[key], np.array([[center[0]], [center[1]], [1]]) )
                new_location = new_location / new_location[2]
                new_image = cv2.circle(new_image, (int(new_location[0]), int(new_location[1])), 5, (255, 0, 0), 5)
                if booleanV == True:
                    new_locations.append( [int(new_location[0]), int(new_location[1])] )
        booleanV = False
        
    show_point_top_view(new_locations)
    
    cv2.imshow("Merged", new_image)
    cv2.waitKey(1)

In [13]:
def read_Image(available_cams):
    
    cam_keys = list(available_cams.keys())

    save_path =  load_dir + '/FinalVideo/top_view_video2.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    top_view_video = cv2.VideoWriter(save_path, fourcc, 25, (600,600))    

    while len(cam_keys) == len(available_cams):
        current_frame = {}        
        for i , key in enumerate(cam_keys):
            is_open , frame = available_cams[key].read()
            if is_open == True:
                current_frame[key] = frame
            else:
                cam_keys.pop(i)
                
        if len(cam_keys) == len(available_cams):
            object_detections = Object_Detection(current_frame)
            top_view_frame = Apply_Homograpy_On_Frames(current_frame)
            label_on_top_view(object_detections, top_view_frame)
            top_view_video.write(top_view_frame)
    
    for key in available_cams.keys():
        available_cams[key].release()
        
    top_view_video.release()

In [14]:
# should run for 6-7 mins so that complete video is stored. If not then the video stored will be incomplete and will
# not open.
available_cams = Get_Available_Cams(cameras, 390)
read_Image(available_cams)
cv2.destroyAllWindows()